In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
# Enable autoreloading if import packages are changed
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from src.utils import utils
import cv2
import glob
import json

PROJECT_DIR = os.getcwd()

# Editable part
dataset_name = "boot_stage2"

### Data Manager

In [ ]:
class DataManager:
    def __init__(self, PROJECT_DIR, dataset_name, dataset_info):
        self.project_dir = PROJECT_DIR
        self.dataset_name = dataset_name
        self.dataset_info = dataset_info
        self.quick_save_reset()
        self._load_data()

    def _load_data(self):
        txt_observation = utils.read_txt(os.path.join(self.project_dir, "dataset", self.dataset_name, "observation.txt"))
        txt_camera_id = utils.read_txt(os.path.join(self.project_dir, "dataset", self.dataset_name, "camera_id.txt"))

        data_observation = np.array([i.strip().split(",") for i in txt_observation], dtype=float)
        self.data_camera_id = np.array([i.strip().split(",") for i in txt_camera_id], dtype=int)
        self.camera_indices = data_observation[:, 0].astype(int)
        self.point_indices = data_observation[:, 1].astype(int)
        self.points_2d = data_observation[:, 2:]
        self.points_3d = np.zeros((len(np.unique(self.point_indices)), 3))
        self.camera_extrinsics = np.array([np.eye(4) for i in range(self.camera_indices.max() + 1)])
        self.camera_intrinsic = np.array(self.dataset_info["parameters"]["camera"]["intrinsics"])

    def get(self):
        return self.data_camera_id, self.camera_indices, self.point_indices, self.points_2d, self.points_3d, self.camera_extrinsics, self.camera_intrinsic

    def set_point_3d(self, idx, p3):
        self.points_3d[idx] = np.array(p3[:3])

    def set_camera_ext(self, idx, cam44):
        self.camera_extrinsics[idx] = np.array(cam44[:4, :4])

    def get_camera_ext(self, idx):
        return self.camera_extrinsics[idx]

    def find_same_points_idx(self, img_src, img_tar):
        img1_pts = self.point_indices[self.camera_indices == img_src] 
        img2_pts = self.point_indices[self.camera_indices == img_tar] 
        intersection = list(set(img1_pts) & set(img2_pts))
        return intersection
    
    def find_same_points_2d(self, img_src, img_tar):
        its = self.find_same_points_idx(img_src, img_tar)
        indexes = np.arange(len(self.point_indices))
        idx_left = np.array([list(set(indexes[self.camera_indices == img_src]) & set(indexes[self.point_indices == pt_idx])) for pt_idx in its]).reshape(-1)
        idx_right = np.array([list(set(indexes[self.camera_indices == img_tar]) & set(indexes[self.point_indices == pt_idx])) for pt_idx in its]).reshape(-1)

        # idx_left =  [np.where(self.point_indices[self.camera_indices == img_src] == idx)[0][0] for idx in its]
        # idx_right = [np.where(self.point_indices[self.camera_indices == img_tar] == idx)[0][0] for idx in its]
        return self.points_2d[idx_left], self.points_2d[idx_right]

    def find_highest_correspondences(self, img_idx, exclude=[]):
        correspondences_count = []
        for i in range(len(self.camera_extrinsics)):
            if (i == img_idx) or (i in exclude):
                correspondences_count.append(0) #skip
                continue
            correspondences_count.append(len(self.find_same_points_idx(img_idx, i)))
        return np.argmax(correspondences_count), correspondences_count
    
    def quick_save_reset(self):
        self.quick_save_record_idx = 0
        self.quick_save_campts_record_idx = 0
    
    def quick_save_data(self, params, num_run):
        savedir = os.path.join(self.project_dir, "dataset", self.dataset_name, "result", f"run{num_run}")
        if not os.path.exists(savedir):
            os.makedirs(savedir)

        savefile = os.path.join(savedir, f"record_{self.quick_save_record_idx}.txt")
        with open(savefile, 'w') as f:
            f.write(params)
        self.quick_save_record_idx += 1

    def quick_save_camera_and_pts3d(self, json_data, num_run):
        savedir = os.path.join(self.project_dir, "dataset", self.dataset_name, "result", f"run{num_run}")
        if not os.path.exists(savedir):
            os.makedirs(savedir)

        savefile = os.path.join(savedir, f"record_{self.quick_save_campts_record_idx}.json")
        utils.export_json(savefile, json_data)
        self.quick_save_campts_record_idx += 1

In [ ]:
dataset_info = utils.get_dataset_info(dataset_name, PROJECT_DIR)
dataset_info["data"]

In [ ]:
data_manager = DataManager(PROJECT_DIR, dataset_name, dataset_info)
data_camera_id, camera_indices, point_indices, points_2d, points_3d, camera_extrinsics, camera_intrinsic = data_manager.get()

### Unit Test

In [ ]:
uv0, uv1 = data_manager.find_same_points_2d(0, 1)

In [ ]:
img_dir = dataset_info["data"]["image"]
img1 = cv2.imread(os.path.join(img_dir, "00000.jpg"))
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
# img2 = cv2.imread(os.path.join(img_dir, "00028.jpg"))
# img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

img_draw1 = img1.copy()#np.hstack((img1, img2)).copy()
img_draw2 = img2.copy()#np.hstack((img1, img2)).copy()

uv_gt1, uv_gt2 = data_manager.find_same_points_2d(0, 1)

for ((u1, v1), (u2, v2)) in zip(uv_gt1, uv_gt2):
    cv2.circle(img_draw1, (int(u1), int(v1)), 1, (0, 255, 0), 10)
    # cv2.circle(img_draw2, (int(u2), int(v2)), 1, (255, 0, 0), 10)

for ((u1, v1), (u2, v2)) in zip(uv_gt1, uv_gt2):
    cv2.line(img_draw1, (int(u1), int(v1)), (int(u2), int(v2)), (255, 255, 0), 2) 
    cv2.circle(img_draw1, (int(u2), int(v2)), 1, (255, 0, 0), 10)


plt.figure(figsize=(25, 15))
plt.imshow(img_draw1)
# plt.figure(figsize=(25, 15))
# plt.imshow(img_draw2)


### Helper Tools definition

In [ ]:
class Tools:
    def __init__(self):
        pass

    @staticmethod
    def reproject_ray(intrinsic, extrinsic, uv):
        # extract oriagin and direction
        origin = extrinsic[:3, 3]
        fx, fy, cx, cy = intrinsic[0, 0], intrinsic[1, 1], intrinsic[0, 2], intrinsic[1, 2]
        norm_coor = np.array([(uv[0] - cx) / fx, - (uv[1] - cy) / fy, 1])
        direction = extrinsic[:3, :3].dot(norm_coor / np.linalg.norm(norm_coor))
        return origin, direction

    @staticmethod
    def reproject_uvs(cam_int, cam_ext, pts_3d):
        cam_ext_inv = cam_ext.copy()
        # cam_ext_inv[:3, :3] = cam_ext_inv[:3, :3].T
        inv_proj = np.linalg.inv(cam_ext_inv)
        uvs = [Tools.reproject_uv(cam_int, inv_proj, pt, True) for pt in pts_3d]
        return uvs      

    @staticmethod
    def reproject_uv(cam_intrinsic, cam_extrinsic, pt_3d, is_extrinsic_inverted=False):
        cam_extrinsic_inv = cam_extrinsic.copy()
        if not is_extrinsic_inverted:
            # inverse extrinsic to project
            # cam_extrinsic_inv[:3, :3] = cam_extrinsic_inv[:3, :3].T
            cam_extrinsic_inv = np.linalg.inv(cam_extrinsic_inv)

        # project to screen
        pt_uv = cam_intrinsic.dot(cam_extrinsic_inv.dot(np.array([pt_3d[0], pt_3d[1], pt_3d[2], 1]))[:3])
        pt_uv = pt_uv/pt_uv[2]

        # if flipping y true
        flip_y = True
        if flip_y:
            pt_uv[1] = 2 * cam_intrinsic[1, 2] - pt_uv[1]
        return pt_uv[:2]

    @staticmethod
    def generate_3d_from_2_cameras(cam_int, cam_ext_src, cam_ext_tar, uv_src, uv_tar, include_dist=False):
        pts_3d, dists = [], []
        for uv_1, uv_2 in zip(uv_src, uv_tar):
            p0, d0 = Tools.reproject_ray(intrinsic=cam_int, extrinsic=cam_ext_src, uv=uv_1)
            p1, d1 = Tools.reproject_ray(intrinsic=cam_int, extrinsic=cam_ext_tar, uv=uv_2)
            pts, dist = utils.triangulate_point(p0, p1, d0, d1)
            dists.append(dist)
            pts_3d.append(pts)
        return np.array(pts_3d), dists if include_dist else None

    @staticmethod
    def recorrect_essential_matrix(cam_int, cam_ext_tar, uv_src, uv_tar):
        # get screen limit
        # screen_width  = 2 * cam_int[0, 2]
        # screen_height = 2 * cam_int[1, 2]

        # # generate 3d
        # pts_3d = Tools.generate_3d_from_2_cameras(cam_int, cam_ext_src, cam_ext_tar, uv_src, uv_tar)

        counts = []
        for i in range(2):
            cam_ext_new = cam_ext_tar.copy()
            # modify translation
            if i % 2 == 1:
                cam_ext_new[:3, 3] = -cam_ext_new[:3, 3]
            # modify rotation
            # if i >= 2:
            #     cam_ext_new[0, 0] = -cam_ext_new[0, 0]
            #     cam_ext_new[1, 1] = -cam_ext_new[1, 1]

            # generate 3d
            pts_3d = Tools.generate_3d_from_2_cameras(cam_int, np.eye(4), cam_ext_new, uv_src, uv_tar)[0]
            pts_3d_cam2 = np.array([np.linalg.inv(cam_ext_new).dot(np.array([p[0], p[1], p[2], 1]))[:3] for p in pts_3d])
            pts_positive_depth = [pt[2] >= 0 for pt in pts_3d]
            pts_positive_depth_cam2 = [pt[2] >= 0 for pt in pts_3d_cam2]
            
            counts.append(np.sum(pts_positive_depth) + np.sum(pts_positive_depth_cam2))            
        print(counts, np.argmax(counts))
        out_cam_ext = cam_ext_tar.copy()
        if np.argmax(counts) % 2 == 1:
            out_cam_ext[:3, 3] = -out_cam_ext[:3, 3]
        # if max(counts) >= 2:
        #     out_cam_ext[0, 0] = -out_cam_ext[0, 0]
        #     out_cam_ext[1, 1] = -out_cam_ext[1, 1]
        return out_cam_ext
        
    # @staticmethod
    # def reproject_uv(cam_intrinsic, cam_extrinsic, pt_3d, is_extrinsic_inverted=False):
    #     cam_extrinsic_inv = cam_extrinsic.copy()
    #     if not is_extrinsic_inverted:
    #         # inverse extrinsic to project
    #         cam_extrinsic_inv = cam_extrinsic.copy()
    #         cam_extrinsic_inv[:3, :3] = cam_extrinsic_inv[:3, :3].T
    #         cam_extrinsic_inv[:3, 3] = -cam_extrinsic_inv[:3, 3]

    #     # project to screen
    #     pt_uv = cam_intrinsic.dot(cam_extrinsic_inv.dot(np.array([pt_3d[0], pt_3d[1], pt_3d[2], 1]))[:3])
    #     pt_uv = pt_uv/pt_uv[2]
    #     return pt_uv[:2]
    
    # @staticmethod
    # def reproject_uvs(cam_int, cam_ext, pts_3d):
    #     inv_proj = np.array(cam_ext)
    #     inv_proj[:3, :3] = inv_proj[:3, :3].T
    #     inv_proj[:3, 3] = -inv_proj[:3, 3]

    #     uvs = [Tools.reproject_uv(cam_int, inv_proj, pt, True) for pt in pts_3d]
    #     return uvs      
    
    @staticmethod
    def rotation_matrix_to_rodriguez(R):
        # R = inv_proj_P2[:3, :3]
        angle = np.arccos((np.trace(R) - 1) / 2) + 1.e-10
        axis = 1 / (2 * np.sin(angle)) * np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

        # Calculate Rodriguez rotation vector
        rodriguez_vector = angle * axis
        return rodriguez_vector

    @staticmethod
    def rodriguez_to_rotation_matrix(rodriguez_vector):
        # Calculate angle of rotation and unit axis vector
        angle = np.linalg.norm(rodriguez_vector)
        axis = rodriguez_vector / angle if angle != 0 else np.array([0, 0, 0])

        # Rodrigues' rotation formula
        skew_symmetric = np.array([[0, -axis[2], axis[1]],
                                [axis[2], 0, -axis[0]],
                                [-axis[1], axis[0], 0]])
        rotation_matrix = np.eye(3) + np.sin(angle) * skew_symmetric + (1 - np.cos(angle)) * np.dot(skew_symmetric, skew_symmetric)
        return rotation_matrix
    

In [ ]:
class Tools_BundleAdjustment:
    def __init__(self):
        pass

    @staticmethod
    def estimate_essentials(camera_intrinsic, uv0, uv1):
        E, mask = cv2.findEssentialMat(uv0, uv1,
                                        method=cv2.RANSAC, 
                                        prob=0.999, 
                                        cameraMatrix=camera_intrinsic)

        R1, R2, t = cv2.decomposeEssentialMat(E)

        # find the combinations
        best_ext, cur_best = None, 0
        for Ri in [R1, R2]:
            for ti in [t, -t]:
                # if ti[0] < 0:
                #     continue
                # ti = t if t[0] > 0 else -t
                # ti = 0.3 * ti
                ext = np.eye(4)
                ext[:3, :3] = Ri.T
                ext[:3, 3] = ti.reshape(-1)
                try:
                    pts_3d = Tools.generate_3d_from_2_cameras(camera_intrinsic, np.eye(4), ext, uv0, uv1)[0]
                    pts_3d_cam2 = np.array([np.linalg.inv(ext).dot(np.array([p[0], p[1], p[2], 1]))[:3] for p in pts_3d])
                except:
                    continue
                total = sum(pts_3d[:, 2] > 0) + sum(pts_3d_cam2[:, 2] > 0)
                if(total > cur_best):
                    best_ext = ext.copy()
                    cur_best = total
        return best_ext

    @staticmethod
    def predict_route(data_manager, max_route=-1):
        camera_indices = data_manager.get()[1]
        img_indexes = np.unique(camera_indices)
        # img_indexes_modified = np.append(img_indexes, img_indexes)[::2]
        known_idx = [0]
        extrinsics_from_essential = {}

        # for each visited image idx, find the best option to take
        while(len(known_idx) != len(img_indexes)):
            best_idx_from = None
            best_idx_to = None
            best_corr_count = 0

            # loop over visited image indexes, then check the highest but unchecked pairs
            # for i, idx_l in enumerate(known_idx):
            #     # correspondences from an image to all
            #     corr_to_all = data_manager.find_highest_correspondences(idx_l)[1]
            #     # then sort and loop over each entry to find whether it still has not yet calculated, max to min
            #     corr_to_all_sorted = np.argsort(corr_to_all)[::-1]
            #     for idx_r in corr_to_all_sorted:
            #         # if already exist, no need to
            #         if img_indexes[idx_r] in known_idx:
            #             # print("skip", idx_l, img_indexes[idx_r])
            #             continue
            #         if corr_to_all[idx_r] > best_corr_count:
            #             best_idx_from = idx_l
            #             best_idx_to = img_indexes[idx_r]
            #             best_corr_count = corr_to_all[idx_r]
            #             # print("best", idx_l, img_indexes[idx_r])
            #             break

            best_idx_from = known_idx[-1]
            best_idx_to = img_indexes[len(known_idx)]

            # img_extrinsics[best_idx_to] = img_extrinsics[best_idx_from]
            print("pose: ", best_idx_from, best_idx_to)
            known_idx.append(best_idx_to)

            img1_2d, img2_2d = data_manager.find_same_points_2d(best_idx_from, best_idx_to)

            # lim_max = 540
            # indexes = np.arange(len(img1_2d))
            # mask = list(set(indexes[img1_2d[:, 1] < lim_max]) and set(indexes[img2_2d[:, 1] < lim_max]))
            # img1_2d, img2_2d = img1_2d[mask], img2_2d[mask]
            # new_ext = ExtrinsicMatrix.essential(pts1=img1_2d, pts2=img2_2d, intrinsic=camera_intrinsic, method="ransac")
            # new_ext = Tools.recorrect_essential_matrix(camera_intrinsic, new_ext, img1_2d, img2_2d)
            new_ext = Tools_BundleAdjustment.estimate_essentials(data_manager.camera_intrinsic, img1_2d, img2_2d)

            extrinsics_from_essential[str([best_idx_from, best_idx_to])] = new_ext
            if not (max_route == -1):
                if (len(known_idx) > 5):
                    break
        return extrinsics_from_essential
    
    @staticmethod
    def rot_y_3x3(th):
        th = np.deg2rad(th)
        return np.array([[np.cos(th), 0, np.sin(th)], [0, 1, 0], [-np.sin(th), 0, np.cos(th)]])

    @staticmethod
    def generate_ext(rot_y, pos):
        ext = np.eye(4)
        ext[:3, :3] = Tools_BundleAdjustment.rot_y_3x3(rot_y)
        ext[:3, 3] = np.array(pos)
        return ext

    # @staticmethod
    # def remap_point_3d(extrinsic_src, extrinsic_tar, pt_3d_src):
    #     inv_ext_src = extrinsic_src.copy()
    #     inv_ext_src[:3, :3] = inv_ext_src[:3, :3].T

    #     pt_3d_h = np.ones(4)
    #     pt_3d_h[:3] = pt_3d_src[:3]

    #     ext_inv = extrinsic_tar.copy()
    #     ext_inv[:3, :3] = ext_inv[:3, :3].T
    #     ext_inv = np.linalg.inv(ext_inv)
    #     return ext_inv.dot(inv_ext_src.dot(np.array(pt_3d_h)))

### Mini-test

In [ ]:
def visualize(cam1_ext, cam2_ext, pt3d):
    cams = [cam1_ext, cam2_ext]

    fixed_camera_origin_direction = [np.zeros(12) for i in cams]
    for j, ext_cam in enumerate(cams):
        # ray = Tools.reproject_ray(camera_intrinsic, ext_cam, (960, 540))
        fixed_camera_origin_direction[j][:3] = ext_cam[:3, 3]
        # fixed_camera_origin_direction[j][3:] = ray[1]
        dir_x, dir_y, dir_z = np.array([1, 0, 0, 1]), np.array([0, 1, 0, 1]), np.array([0, 0, 1, 1])
        fixed_camera_origin_direction[j][3:6] = ext_cam.dot(dir_x)[:3]
        fixed_camera_origin_direction[j][6:9] = ext_cam.dot(dir_y)[:3]
        fixed_camera_origin_direction[j][9:12] = ext_cam.dot(dir_z)[:3]
    fixed_camera_origin_direction = np.array(fixed_camera_origin_direction)

    ax = plt.figure(figsize=(15, 10)).add_subplot(projection='3d')

    color = ["r", "g", "b"]
    # training - scatter pt and cam
    ax.scatter(xs=pt3d[:, 0], ys=pt3d[:, 1], zs=pt3d[:, 2], zdir='y', label='Known pts', c='magenta')
    ax.scatter(xs=fixed_camera_origin_direction[:, :3][:, 0], ys=fixed_camera_origin_direction[:, :3][:, 1], zs=fixed_camera_origin_direction[:, :3][:, 2], zdir='y', label='Known camera', c='g')

    # known - plot cam
    for cam_data_fixed in fixed_camera_origin_direction:
        for k_idx, k in enumerate(cam_data_fixed[3:].reshape(3, 3)):
            start = cam_data_fixed[:3]
            end = k / np.linalg.norm(k - start)# / np.linalg.norm(k)#cam_data[:3] + cam_data[3:]
            # end = cam_data[:3] + cam_data[3:6]
            poses = np.array([start, end]).reshape(-1, 3)
            ax.plot(xs=poses[:, 0], ys=poses[:, 1], zs=poses[:, 2], zdir="y", c=color[k_idx])

    ax.view_init(elev=45., azim=-45, roll=0)
    ax.set_xlabel("X")
    ax.set_ylabel("Z")
    ax.set_zlabel("Y")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-1, 5)
    ax.set_zlim(0, 5)

    ax.legend()

In [ ]:
is_generate_route_from_data_manager = True

route = {}
if is_generate_route_from_data_manager:
    route = Tools_BundleAdjustment.predict_route(data_manager)
else:
    # load directly from file correspondences
    img_idx = np.array([int(os.path.basename(i).split(".")[0]) for i in glob.glob(os.path.join(dataset_info["data"]["image"], "*.jpg"))])
    img_idx = sorted(img_idx)
    extrinsics_from_essential = {}

    start_idx = 0
    for i, img_name_l in enumerate(img_idx[start_idx:-1]):
        p1, p2 = [], []
        img_name_r = img_idx[start_idx + i + 1]
        corr_l_exist, corr_r_exist, corr_exist = utils.is_correspondence_exist(PROJECT_DIR, dataset_name, img_name_l, img_name_r)
        correspondences_path = ""
        if corr_exist:
            if corr_l_exist:
                correspondences_path = os.path.join(dataset_info["data"]["correspondence"], f"{img_name_l}_{img_name_r}.txt")
                corr_data = utils.read_txt(correspondences_path)
                for k, corr in enumerate(corr_data):
                    p1_x, p1_y, p2_x, p2_y = np.array(corr.rstrip().split(" ")).astype(float)
                    p1.append([p1_x, p1_y])
                    p2.append([p2_x, p2_y])
            elif corr_r_exist:
                correspondences_path = os.path.join(dataset_info["data"]["correspondence"], f"{img_name_r}_{img_name_l}.txt")
                corr_data = utils.read_txt(correspondences_path)
                for k, corr in enumerate(corr_data):
                    p2_x, p2_y, p1_x, p1_y = np.array(corr.rstrip().split(" ")).astype(float)
                    p1.append([p1_x, p1_y])
                    p2.append([p2_x, p2_y])
        p1 = np.array(p1)
        p2 = np.array(p2)
        
        new_ext = Tools_BundleAdjustment.estimate_essentials(data_manager.camera_intrinsic, p1, p2)

        extrinsics_from_essential[str([start_idx + i, start_idx + i+1])] = new_ext
        route = extrinsics_from_essential

In [ ]:
run_idx = 2

In [ ]:
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares, Bounds
import time

class BundleAdjustment:
    def __init__(self, data_manager: DataManager, sequence_essentials: dict, start_cam=0):
        self.data_manager = data_manager
        self.sequence_essentials = sequence_essentials
        self.start_cam = start_cam
        self.known_extrinsics = {start_cam: np.eye(4)}
        self.known_3d_points = {}
        self.exclude_triangulation = {} #{"[cam_from, cam_to]": [idxs]}
        self.cam_from = 0
        self.counter = -1
        self.interval = 250
        self.max_num_camera_each_loop = 5

    def run(self, iteration_idx=0):
        # default parameters
        n_params_camera = 6
        n_params_point = 3
        route = list(self.sequence_essentials.keys())
        _, camera_indices, point_indices, points_2d, _, _, camera_intrinsic = self.data_manager.get()
        

        for step in range(2):
        # if True:
            # step = 0
            # first one for the 2 camera iteration, second for all cameras refinemenet

            points_ids = [] #store intersection point indexes
            camera_ids = [] #store camera indexes to train
            
            if (step == 0):
                # 1st step: calibrate only using 2 cameras. define number of cameras and points
                # find the same points between two cameras 
                print(f" ----- Start iteration {iteration_idx}, camera pair: {route[iteration_idx]} ----- ")
                cam_from, cam_to = np.array(route[iteration_idx][1:-1].split(","), dtype=int)
                self.cam_from = cam_from
                its = self.data_manager.find_same_points_idx(cam_from, cam_to)
                camera_ids = [cam_from, cam_to]
                points_ids = its
            elif (step == 1):
                # if ((iteration_idx - 1) % 4 != 0):
                #     continue
                
                # 2nd step: refine all cameras
                print(f" ----- Refine cameras and points, camera pairs: {list(self.known_extrinsics.keys())} ----- ")
                # check known cameras, and known points
                if not len(self.known_extrinsics.keys()) % self.max_num_camera_each_loop == 0:
                    camera_ids = list(self.known_extrinsics.keys())[-self.max_num_camera_each_loop:]
                else:
                    camera_ids = list(self.known_extrinsics.keys())
                points_ids = list(self.known_3d_points.keys())
                self.cam_from = camera_ids[0]
                # self.cam_from = self.start_cam
            else:
                print(f"Step{step} not known!")

            # loop over indices to fill the arrays
            camera_indices_filter = []
            point_indices_filter = []
            points_2d_filter = []
            points_3d_filter = []
            for cam, pt, pt2 in zip(camera_indices, point_indices, points_2d):
                if (pt in points_ids) and (cam in camera_ids):
                    # exclude marked point
                    if (cam in list(self.exclude_triangulation.keys())):
                        if (pt in list(self.exclude_triangulation[cam])):
                            continue
                    camera_indices_filter.append(cam)
                    point_indices_filter.append(pt)
                    points_2d_filter.append(pt2)
            camera_indices_filter = np.array(camera_indices_filter)
            point_indices_filter = np.array(point_indices_filter)
            points_2d_filter = np.array(points_2d_filter)
            points_3d_filter = np.array(points_3d_filter)

            # change the point indices into different id (for training and conversion later)
            # create mapping dictionary
            unique_points_id_sorted = np.unique(point_indices_filter)
            unique_points_mapping_dict = dict([(k, v) for k, v in zip(unique_points_id_sorted, np.arange(len(unique_points_id_sorted)))])
            unique_points_mapping_dict_reverse = dict([(v, k) for k, v in unique_points_mapping_dict.items()])
            point_indices_filter_2 = np.copy(point_indices_filter)
            for k, v in unique_points_mapping_dict.items(): point_indices_filter_2[point_indices_filter==k] = v

            # change the camera indices too
            unique_camera_id_sorted = np.unique(camera_indices_filter)
            unique_camera_mapping_dict = dict([(k, v) for k, v in zip(unique_camera_id_sorted, np.arange(len(unique_camera_id_sorted)))])
            unique_camera_mapping_dict_reverse = dict([(v, k) for k, v in unique_camera_mapping_dict.items()])
            camera_indices_filter_2 = np.copy(camera_indices_filter)
            for k, v in unique_camera_mapping_dict.items(): camera_indices_filter_2[camera_indices_filter==k] = v

            #  ---- fixed! ---

            # and the rest
            camera_extrinsics_filter = [] #0th camera as default
            points_3d_filter = np.random.random((len(unique_points_id_sorted), 3))
            if (step == 0):
                # calibrate the 3d points, if the points has been intersected before,
                # put it directly into the array
                cam_from, cam_to = np.array(route[iteration_idx][1:-1].split(","), dtype=int)

                # NEW Camera! only 2 cameras calibration
                # new_extrinsics = self.sequence_essentials[route[iteration_idx]].copy().dot(self.known_extrinsics[cam_from].copy())
                new_extrinsics = self.known_extrinsics[cam_from].dot(self.sequence_essentials[route[iteration_idx]])
                camera_extrinsics_filter.append(self.known_extrinsics[cam_from])
                camera_extrinsics_filter.append(new_extrinsics)

                # find same points between two image index
                # its = list(unique_points_id_sorted)#self.data_manager.find_same_points_idx(cam_from, cam_to)
                its = self.data_manager.find_same_points_idx(cam_from, cam_to)
                
                its_known = list(set(its) & set(list(self.known_3d_points.keys())))
                for i in its_known:
                    # find its index (idx) on the current dict 
                    # print(i, "\n", its, "\n", its_known, "\n", unique_points_mapping_dict)
                    # TODO: For every known 3d points, map it wrt the cam_from since we will apply it 
                    idx = unique_points_mapping_dict[i] #get arange index

                    # transform the known homogenous point
                    # points_3d_filter[idx] = Tools_BundleAdjustment.remap_point_3d(self.known_extrinsics[0], new_extrinsics, self.known_3d_points[i].copy())[:3]
                    points_3d_filter[idx] = self.known_3d_points[i]["pos"].copy()
                
                # look into the intersections that is now known, find the index that is not known from its
                its_not_known = list(set(its) - set(self.known_3d_points.keys()))
                idx_not_known_from_its = [unique_points_mapping_dict[i] for i in its_not_known]
                # idx_not_known_from_its = [its.index(i) for i in its_not_known]

                if len(idx_not_known_from_its) > 0:
                    # store uv and 3d points
                    uv_src, uv_tar = self.data_manager.find_same_points_2d(cam_from, cam_to)    
                    uv_src, uv_tar = uv_src[idx_not_known_from_its], uv_tar[idx_not_known_from_its]
                    pts_3d = Tools.generate_3d_from_2_cameras(cam_int=camera_intrinsic, 
                                                            cam_ext_src=self.known_extrinsics[cam_from],
                                                            cam_ext_tar=new_extrinsics,
                                                            uv_src=uv_src,
                                                            uv_tar=uv_tar)[0]

                    # store points 3d filter
                    # points_3d_filter = pts_3d
                    points_3d_filter[idx_not_known_from_its] = pts_3d

            elif (step == 1):
                # combine all existing extrinsics
                camera_extrinsics_filter = np.array([self.known_extrinsics[idx] for idx in unique_camera_id_sorted])
                # camera_extrinsics_filter = np.array([list(self.known_extrinsics.values())])

                # get all known 3d points
                # points_3d_filter = np.array([self.known_3d_points[idx] for idx in unique_points_id_sorted])
                points_3d_filter = np.array([self.known_3d_points[idx]["pos"] for idx in unique_points_id_sorted])
                

            # count number of observations
            # number of camera always -1 since we assume that the first camera extrinsics is identity
            # ignore it for now
            n_cameras = len(unique_camera_id_sorted)
            n_points = len(unique_points_id_sorted)
            n_observation = 2 * len(point_indices_filter_2)
            n_variable = n_params_camera * n_cameras + n_params_point * n_points
            print(f" using {n_cameras} cameras and {n_points} points")

            # --- start build sparsity matrix ---
            A = lil_matrix((n_observation, n_variable), dtype=float)

            # fill the matrix with ones
            i = np.arange(int(0.5 * n_observation))
            for s in range(6):
                A[2 * i,     camera_indices_filter_2 * n_params_camera + s] = 1
                A[2 * i + 1, camera_indices_filter_2 * n_params_camera + s] = 1

            for s in range(3):
                A[2 * i,     n_cameras * n_params_camera + point_indices_filter_2 * n_params_point + s] = 1
                A[2 * i + 1, n_cameras * n_params_camera + point_indices_filter_2 * n_params_point + s] = 1
            # --- end build sparsity matrix ---

            # --- build input x ---
            x = []
            
            camera_extrinsics_filter = np.array(camera_extrinsics_filter)
            points_3d_filter = np.array(points_3d_filter)
            for cam_ext in camera_extrinsics_filter:
                pos = cam_ext[:3, 3]
                rot = Tools.rotation_matrix_to_rodriguez(cam_ext[:3, :3])
                x.extend(pos)
                x.extend(rot)      
            for pt in points_3d_filter:
                x.extend(pt)
            # --- end build input x ---
            # print(len(x), n_cameras, n_points, n_observation, n_variable)
            # print(len(camera_extrinsics_filter), camera_extrinsics_filter)
            # print(len(points_3d_filter), points_3d_filter)
            
            # --- build trainer ---
            t0 = time.time()
            #jac="3-point", xtol=1e-5, gtol=1e-5
            ftol_value = 1e-4 #if n_cameras == 2 else 1e-5
            res = least_squares(self._fun, x, jac_sparsity=A, verbose=2, x_scale='jac', ftol=ftol_value, method='dogbox', loss="soft_l1",
                                args=(n_cameras, n_points, camera_intrinsic, camera_indices_filter_2, point_indices_filter_2, 
                                      points_2d_filter))
            t1 = time.time()
            print(f" done in {t1-t0}s")
            # --- end trainer ---

            # --- store data ---
            params = res.x
            cam_params = params[:n_cameras * 6].reshape((n_cameras, 6))
            points_3d = params[n_cameras * 6:].reshape((n_points, 3))
            if (step == 0):
                # # extract idx, just two of them
                cam_from, cam_to = unique_camera_id_sorted
                # # multiply ext result with stored camera extrinsics of the reference image (left)
                # cam_mat = np.eye(4)
                # cam_mat[:3, 3] = cam_params[1][:3]
                # cam_mat[:3, :3] = Tools.rodriguez_to_rotation_matrix(cam_params[1][3:])
                # self.known_extrinsics[cam_to] = cam_mat.dot(self.known_extrinsics[cam_from])
                for cam, id in zip(cam_params, unique_camera_id_sorted):
                    cam_mat = np.eye(4)
                    cam_mat[:3, 3] = cam[:3]
                    cam_mat[:3, :3] = Tools.rodriguez_to_rotation_matrix(cam[3:])
                    self.known_extrinsics[id] = cam_mat

                # add outlier rejection system
                uvs_gt = self.data_manager.find_same_points_2d(cam_from, cam_to)
                _, dists = Tools.generate_3d_from_2_cameras(camera_intrinsic, self.known_extrinsics[cam_from], self.known_extrinsics[cam_to], uvs_gt[0], uvs_gt[1], include_dist=True)  
                dists_sorted = np.sort(dists)
                
                # ratio = 0.5
                ratio = (1.58 - 0.0036 * len(points_3d))
                ratio = max(min(0.9, ratio), 0.5)
                limit = min(np.average(dists_sorted), dists_sorted[int(ratio * len(dists_sorted))])
                bool_outlier = dists > limit

                for pt_idx, (pt_3d, is_outlier) in enumerate(zip(points_3d, bool_outlier)):
                    # get true point index from the dict
                    true_pt_idx = unique_points_mapping_dict_reverse[pt_idx]

                    # skipping the outlier for the refinement in the 2nd step
                    if is_outlier:
                        c = cam_from
                        if c not in self.exclude_triangulation.keys():
                            self.exclude_triangulation[c] = []
                        self.exclude_triangulation[c].append(true_pt_idx)
                        print(f"skip a {pt_idx}, dists {dists[pt_idx]}")
                        continue

                    # self.known_3d_points[true_pt_idx] = pt_3d[:3]  
                    if true_pt_idx in list(self.known_3d_points.keys()):
                        # compare the distance before saving
                        if dists[pt_idx] < self.known_3d_points[true_pt_idx]["dist"]:
                            self.known_3d_points[true_pt_idx] = {
                                "pos": pt_3d[:3], "dist": dists[pt_idx]
                            }
                        else:
                            # skip saving the point if the new distance is larger
                            continue
                    else:                        
                        self.known_3d_points[true_pt_idx] = {
                            "pos": pt_3d[:3], "dist": dists[pt_idx]
                        }

            elif (step == 1):
                # just save it
                # for cam_idx, cam_param in zip(unique_camera_id_sorted, cam_params): #ignore 0th
                for cam_idx, cam_param in enumerate(cam_params): #ignore 0th
                    cam_mat = np.eye(4)
                    cam_mat[:3, 3] = cam_param[:3]
                    cam_mat[:3, :3] = Tools.rodriguez_to_rotation_matrix(cam_param[3:])
                    self.known_extrinsics[unique_camera_mapping_dict_reverse[cam_idx]] = cam_mat
                
                # readjust known 3d points entirely
                cam_ext_indexes = list(self.known_extrinsics.keys())
                for cam_from_idx in range(len(cam_ext_indexes) - 1):
                    c_from, c_to = cam_ext_indexes[cam_from_idx], cam_ext_indexes[cam_from_idx + 1]
                    its = self.data_manager.find_same_points_idx(c_from, c_to)
                    pts_2d_from, pts_2d_to = self.data_manager.find_same_points_2d(c_from, c_to)
                    pts_3d_new, dists = Tools.generate_3d_from_2_cameras(camera_intrinsic, self.known_extrinsics[c_from], self.known_extrinsics[c_to], pts_2d_from, pts_2d_to, include_dist=True)  

                    for (k, pt_3d, dist) in zip(its, pts_3d_new, dists):
                        if k in self.exclude_triangulation[c_from]:
                            # skip if the point is marked before to not included
                            continue

                        # save the point and distance
                        self.known_3d_points[k] = {
                            "pos": pt_3d[:3], "dist": dist
                        }


                # for pt_idx, pt_3d in enumerate(points_3d):
                #     # if np.linalg.norm(pt_3d) > 10:
                #     #     print(f"skip b {pt_idx}")
                #     #     continue
                #     true_pt_idx = unique_points_mapping_dict_reverse[pt_idx]
                #     self.known_3d_points[true_pt_idx] = pt_3d[:3]

        # --- end store data ---
        return res, (t1-t0)

    def _fun(self, params, n_cameras, n_points, camera_intrinsic, camera_indices, point_indices, points_2d):
        "Compute residual (error) projection for each observation"
        # extract camera parameters
        cam_params = params[:n_cameras * 6].reshape((n_cameras, 6))
        points_3d = params[n_cameras * 6:].reshape((n_points, 3))

        # save record
        self.counter += 1
        self.counter = self.counter % self.interval
        save_record = True
        if save_record and (self.counter == 0):
            # print(self.counter, self.data_manager.quick_save_record_idx, self.data_manager.quick_save_campts_rec|ord_idx)
            rec = [len(cam_params), len(points_3d)]
            rec.extend(params)
            self.data_manager.quick_save_data(str(rec), run_idx)
            self.data_manager.quick_save_camera_and_pts3d({
                "camera": [list(i.flatten()) for i in self.known_extrinsics.values()],
                "pts": [list(i["pos"]) for i in self.known_3d_points.values()]
            }, run_idx)

        ratio_distance = 1    
        # if self.cam_from == 0:
        #     # it means full adjustment
        #     ratio_distance = np.linalg.norm(cam_params[1][:3])
        # else:
            # only two cameras
            # ratio_distance = 1 / np.linalg.norm(cam_params[1][:3] - self.known_extrinsics[self.cam_from][:3, 3])
        cam_extrinsics = [self.known_extrinsics[self.cam_from]] # always ignore the 0th camera 
        # cam_extrinsics = [] # not ignore the 0th camera 
        for cam in cam_params[1:]:
            cam_mat = np.eye(4)
            cam_mat[:3, 3] = cam[:3] * ratio_distance
            cam_mat[:3, :3] = Tools.rodriguez_to_rotation_matrix(cam[3:])
            cam_extrinsics.append(cam_mat)


        uvs = []
        for cam_idx, pt_idx in zip(camera_indices, point_indices):
            # try reiterate the point (maybe it works? dunno)
            # xs, ys, zs = [-10, 10], [-10, 10], [-10, 10]
            # if ((points_3d[pt_idx][0] < xs[0]) or (points_3d[pt_idx][0] > xs[1])) or \
            #     ((points_3d[pt_idx][1] < ys[0]) or (points_3d[pt_idx][1] > ys[1])) or \
            #     ((points_3d[pt_idx][2] < zs[0]) or (points_3d[pt_idx][2] > zs[1])):
            #     points_3d[pt_idx] = np.random.uniform(low=-10, high=10, size=(3))

            uv = Tools.reproject_uv(cam_intrinsic=camera_intrinsic, 
                               cam_extrinsic=cam_extrinsics[cam_idx],
                               pt_3d=points_3d[pt_idx],
                               is_extrinsic_inverted=False)
            uvs.append(uv)
        # uvs = np.array(uvs).astype(int)
        # points_2d = points_2d.astype(int)
        diff = np.abs(uvs - points_2d).ravel()
        # for i, (uv1, uv2) in enumerate(zip(uvs, points_2d)):
        #     if i > 3: 
        #         break
        #     print(uv1, uv2, diff.reshape(-1, 2)[i], diff.sum(), points_3d[i])
        # print()
        return diff


In [ ]:
start_idx_ba = 0
run_idx = 2
data_manager.quick_save_reset()
bundle_adjust = BundleAdjustment(data_manager, route, start_idx_ba)

In [ ]:
for i in range(len(bundle_adjust.known_extrinsics.keys())-1 + start_idx, len(route) + start_idx):
# for i in range(start_idx_ba, len(route) + start_idx_ba):
# for i in range(start_idx_ba, 6 + start_idx_ba):
    res = bundle_adjust.run(i)

    # save data
    dataset_path = dataset_info["data"]["path"]
    new_known_3d_points = {}
    for k, v in bundle_adjust.known_3d_points.items():
        new_known_3d_points[f"p{k}"] = v#list(v)
        new_known_3d_points[f"p{k}"]["pos"] = list(new_known_3d_points[f"p{k}"]["pos"])
    utils.export_json(f"{dataset_path}/result/run{run_idx}/known_3d_points_{i}.json", new_known_3d_points)

    new_known_extrinsics = {}
    for k, v in bundle_adjust.known_extrinsics.items():
        new_known_extrinsics[f"i{data_camera_id[k][1]}"] = list(v.flatten())
    utils.export_json(f"{dataset_path}/result/run{run_idx}/known_camera_extrinsics_{i}.json", new_known_extrinsics)